# Tutorial ForkTPS solver
In this first tutorial, we use the ForkTPS solver for a non-interacting one orbital model with different semi-circular bath DOS for the spin-up and the spin-down block. Even without interactions this is a non-trivial case for tensor networks, since non-interacting systems are entangled in general.

Here we are not concerned with the approximation parameters of the calculation and just use the default values. These will be covered in another tutorial.

In [ ]:
#%matplotlib notebook
import forktps as ftps
import matplotlib.pyplot as plt

import numpy as np

from forktps.DiscreteBath import *
from forktps.Helpers import *

from triqs.gf import *

## First lets initialize a bath hybridization $\Delta(\omega)$ for a simple one-orbital model

Note that the names for the block are restricted to "up" and "dn" or "down".

In [ ]:
Norbs = 1
gfstruct = [['up',[0]], ['dn',[0]]]

dup = GfReFreq(indices = [0], window = (-3., 3.), n_points =  3001, name = "up")
ddn = GfReFreq(indices = [0], window = (-3., 3.), n_points =  3001, name = "dn")

for i in range(Norbs):
    dup[i,i] << SemiCircular(half_bandwidth = 1.3)
    ddn[i,i] << SemiCircular(half_bandwidth = 2.0)

Delta = BlockGf(name_list = ('up','dn'), block_list = (dup,ddn), make_copies = False)

## Next let us construct a discretized bath from $\Delta(\omega)$

For this DiscreteBath.py provides a function called 'DiscretizeBath'. It takes a hybridization function $\Delta(\omega)$ and the number of bath sites and returns a bath-object containing the bath parameters.

Important: Nb = 9 means that we use 9 bath sites for each orbital (and each spin). For the two orbital model simulated in this tutorial, we hence have a total number of $2\cdot(9+1) = 20$ spinless fermion degrees of freedom or equivalently $10$ Hubbard sites (counting also the impurity). This is already not far from the limit what an exact diagonalization solver usually can do.

In [ ]:
Nb = 9 #number of bath sites
bath = DiscretizeBath(Delta=Delta, Nb=Nb) #bath object storing all relevant information for the bath
eta = 0.1

#### Important: It is good practice to check whether the bath just created is a satisfactory approximation to the true $\Delta(\omega)$
So let us calculate the hybridization from the bath parameters: $\sum_k \frac{V_k^2}{\omega - \epsilon_k + i \eta}$ for some finite broadening $\eta$ (note this formula is valid for diagonal hybridizations).
A good choice of the value of $\eta$ is the energy spacing between to consecutive bath energies.
<br><br>
_Note that the reconstructed hybridization plotted here is not a physical quantity. The discrete bath is in fact a sum of delta-peaks. In order to get a visual understanding of how well these delta-peaks approximate the true hybridization, we plot it with some finite broadening eta._


In [ ]:
DeltaDiscrete = bath.reconstructDelta( w = ddn.mesh, eta = eta )

plt.figure()
for i, [name, g] in enumerate(Delta):
    plt.subplot(1,2,i+1)
    plt.title(name)
    for j in range(g.target_shape[0]):
        plt.plot( getX(Delta.mesh), -Delta[name][j,j].imag.data, label = 'Original')
        plt.plot( getX(Delta.mesh), -DeltaDiscrete[name][j,j].imag.data, label = 'Reconstruction')
    plt.legend()

## Next, let us fix the remaining parameters of the model, the local non-interacting Hamiltonian of the Impurity and the interaction
Currently, the interaction only Kanamori type Hamiltonians are implemented with- and without spinflip and pair hopping terms

In [ ]:
e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill( [[0]] ) # e0 is a matrix hence the double brackets

# Interaction parameters U, Hund's Coupling J and inter-orbital U'. Additionally there is a flag if the calculation should be performed with density-density terms only (dd=True) or with including spin-flip and pair-hopping terms (dd=False). 
Hint = ftps.solver_core.HInt(u=0.,j=0., up=0., dd = True)

## Now that the model is set up, we can initialize and run the Solver 
We have to add the bath and the local Hamiltonian to the solver itself whereas the interaction is provided at solve call. This is to mimic the behavior of CTQMC solver that use the Weiss-field $G_0$ as solver member and also need the interaction at solve-call.

Note that this calculation can take a little while, because with the standard settings used in this tutorial all possible Green's functions are calculated on after the other.

In [ ]:
S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-3., wmax=3.)
# TRIQS solver often have the Weiss field as member providing information about the bath. 
# The bath and e0 carry the exact same information for the ForkTPS solver.
S.b = bath
S.e0 = e0

# Set only necessary paramters, dt is the time step during time evolution and time_steps is the number of time steps taken in a single time evolution (t_max of the Greens function is then 2*time_steps*dt because Green's function are calculated from two time evolutions)
tevoParams = ftps.solver.TevoParams(dt = 0.5, time_steps=20)
S.solve( h_int = Hint, tevo = tevoParams, eta = eta)

# Let us take a look at some quantities
### Greater Green's function:
DiscreteBath.py provides functions to calculate the exact greater and lesser Green's function for a non-interacting bath, so let's compare the result of the solver with the exact solution.

In [ ]:
Ggr_exact = CalcGgr(bath=bath, hloc=e0, mesh=S.G_gr.mesh )
t = getX(S.G_gr.mesh)

for name, g in Ggr_exact:
    plt.figure(figsize = (8,4))
    plt.title(name)
    for j in range(g.target_shape[0]):
        plt.subplot(1,2,1)
        plt.suptitle("Block: " + name)
        plt.title(r"$\Re G^>(t) $" )
        plt.plot( t, Ggr_exact[name][j,j].real.data, label = f'Exact indx {j}' )
        plt.plot( t, S.G_gr[name][j,j].real.data,    label = f'Solver indx {j}', marker = 'o', ls = '--' )
        plt.ylabel("$G(t)$")
        plt.legend()
        
        plt.subplot(1,2,2)
        plt.title("Difference" )
        difference = Ggr_exact[name][j,j].real.data -S.G_gr[name][j,j].real.data
        plt.plot( t, difference, label = f'Difference indx {j}')
        plt.legend()

## What can we observe? 
The error is very small, of the order $10^{-7}$. Remember for tensor networks, non-interacting models are non-trivial, so this is a very accurate result.
<br> <br>
Both Green's functions decay at first, but start to slowly grow again at some time.
Why is that? As mentioned above, the bath that we use in the ForkTPS solver is discrete, i.e, it consists of delta-peaks in energy space. Using a Fourier transform to go from energy space to real-times we have to Fourier transform delta-peaks resulting in functions that osciallate indefinetly.
<br> <br>
A more physically intuitive picture can be obtained by thinking of the Impurity problem in the Wilson chain geometry where the bath is essentially a tight binding chain and the impurity couples only to the first site. During the calculation of the Green's function we add a particle to the impurity. This particle travels outwards the chain and will hit the end upon which it is reflected and travels back towards the impurity. For the spin-down block, the particle comes back to the impurity at time $t=22$ (peak in $G(t)$). This means that around $t=11$, it hits the end of the chain. We observe, that at the latter time also the error fluctuates strongly.
<br> <br>
From the plots, it looks like the spin-down block shows this finite size effect earlier than the spin-up block. Can you figure out why?

# Spectral Function
Finally, let us take a look at the spectral function. Since for no interactions $G(\omega) = G_0(\omega) = \frac{1}{\omega - \Delta(\omega) + i \eta}$, we can make two comparisons. We can compare either to the exact solution obtained from the discrete or from the continuous hybridization.

Note that using a finite broadening for the Green's function obtained from the continuous bath is not 100% correct. One actually would have to broaden $\Delta(\omega)$ as well by a convolution with a Lorentzian of width $\eta$. For now let us neglect this though.

We find that the spin-down spectral function seems to reproduce the bath spectrum. This is because the width and height of the bath spectrum are such that we simulate the self-consistent hybridization of the non-interacting Bethe lattice.

Again, we find that the discrete bath and the solver output agree very well. The remaining difference between these two is due to the finite maximum time for which we calculated the Green's function. Comparing to the continuous hybridization though reveals some differences due to not representing the bath well enough. Still, in this first tutorial we only used a very small number of bath sites and the agreement to the true Green's function is at least acceptable.

In [ ]:
w = getX(Delta.mesh)
G_w_ex_discr = BlockGf(mesh=Delta.mesh, gf_struct = gfstruct)
G_w_ex = BlockGf(mesh=Delta.mesh, gf_struct = gfstruct)


for name, g in G_w_ex:
    G_w_ex_discr[name] << inverse( Omega + 1j*eta - S.Delta_recons_w[name] )
    G_w_ex[name]       << inverse( Omega + 1j*eta - Delta[name] )

plt.figure(figsize = (8,4))
for i, [name, g] in enumerate(S.G_w):
    plt.subplot(1, 2,i+1)
    plt.title(name)
    plt.plot( w, -g[0,0].imag.data ,                 label = 'Solver'   )
    plt.plot( w, -G_w_ex_discr[name][0,0].imag.data, label = 'Discrete Bath' )
    plt.plot( w, -G_w_ex[name][0,0].imag.data,       label = 'Continuous Bath' )
    plt.ylabel("$A(\omega)$")
    plt.legend()
    
